In [1]:
# Google ColabやLinux環境の場合
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -
!echo 'deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main' | tee /etc/apt/sources.list.d/google-chrome.list
!apt-get update
!apt-get install google-chrome-stable
!pip install selenium webdriver-manager

OK
deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://cli.github.com/packages stable InRelease [3,917 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Get:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [4,009 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [70.9 kB]
Get:13 http://archive.ubu

In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
import datetime
from datetime import timedelta
import re

# ==========================================
# Step 2. HTMLから株価データを抽出する関数
# ==========================================
def extract_stock_data(html):
    soup = BeautifulSoup(html, 'html.parser')
    extracted_data = {}
    keywords = ['始値', '高値', '安値', '終値']

    # 日付検索
    text_content = soup.get_text(separator=' ', strip=True)
    date_match = re.search(r'(202\d/\d{1,2}/\d{1,2})', text_content)

    if not date_match:
        return None

    date_str = date_match.group(1)

    # 数値検索
    for key in keywords:
        elements = soup.find_all(string=re.compile(key))
        for el in elements:
            parent = el.parent
            if not parent: continue

            target_text = parent.get_text(strip=True)
            if not re.search(r'\d', target_text.replace(key, '')):
                for sibling in parent.find_next_siblings(limit=2):
                    target_text += " " + sibling.get_text(strip=True)

            num_match = re.search(r'[\d,]+\.\d+|[\d,]+', target_text.replace(key, ''))
            if num_match:
                extracted_data[key] = num_match.group(0)
                break

    if all(k in extracted_data for k in keywords):
        try:
            dt = datetime.datetime.strptime(date_str, '%Y/%m/%d')
            formatted_date = dt.strftime('%Y-%m-%d')
            return [
                formatted_date,
                extracted_data['始値'],
                extracted_data['高値'],
                extracted_data['安値'],
                extracted_data['終値']
            ]
        except Exception:
            return None
    return None

# ==========================================
# Step 3. Webページへアクセスしデータを抽出する関数
# ==========================================
def get_stock_values(driver, url):
    print("サイトにアクセスしています...")
    driver.get(url)
    time.sleep(5)

    try:
        chart_element = driver.find_element(By.CSS_SELECTOR, '.highcharts-container')
    except:
        print("エラー: チャート要素が見つかりません。")
        return []

    # チャートが見える位置までスクロール
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", chart_element)
    time.sleep(2)

    # チャート情報取得
    chart_size = chart_element.size
    width = chart_size['width']
    height = chart_size['height']

    actions = ActionChains(driver)

    # スタート地点（右端）
    start_x = width * 0.96
    start_y = height * 0.4

    actions.move_to_element_with_offset(chart_element, start_x, start_y)
    actions.perform()
    time.sleep(1)

    stock_data_list = []
    seen_dates = set()

    # --- 【修正点】日付ベースの終了条件を設定 ---
    # 今日から約185日前（6ヶ月分）を計算
    today = datetime.date.today()
    limit_date = today - timedelta(days=185)

    print(f"データ取得開始: {limit_date} 以前のデータに到達するまで遡ります。")
    print("時間がかかります（目標: 120件以上）...")

    # 最大ループ回数（無限ループ防止用）
    max_loops = 1000

    for i in range(max_loops):
        html = driver.page_source
        data = extract_stock_data(html)

        current_date_obj = None

        if data:
            date_str = data[0]
            # 日付文字列をオブジェクトに変換して比較用にする
            current_date_obj = datetime.datetime.strptime(date_str, '%Y-%m-%d').date()

            if date_str not in seen_dates:
                stock_data_list.append(data)
                seen_dates.add(date_str)

                # 進捗ログ（10件ごとに表示）
                if len(stock_data_list) % 10 == 0:
                    print(f"  -> {len(stock_data_list)}件取得 (現在: {date_str})")

        # --- 終了判定 ---
        # 取得した日付が「6ヶ月前」より古くなったら終了
        if current_date_obj and current_date_obj < limit_date:
            print("6ヶ月分のデータ取得が完了しました。")
            break

        # マウス移動（左へ）
        # ステップ幅を少し調整（早すぎると読み飛ばすが、遅すぎると終わらない）
        try:
            actions.move_by_offset(-4, 0)
            actions.perform()
        except:
            print("画面端に到達しました。")
            break

        # 待機時間
        time.sleep(0.25)

    return stock_data_list

# ==========================================
# Step 4. mainコード
# ==========================================
if __name__ == "__main__":
    start_time = time.time()

    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    driver.implicitly_wait(10)

    target_url = "https://www.nikkei.com/markets/worldidx/chart/nk225/?type=6month"

    try:
        result_data = get_stock_values(driver, target_url)

        end_time = time.time()

        print("-" * 40)
        print(f"スクレイピングにかかった時間: {end_time - start_time:.2f}秒")
        print(f"取得件数: {len(result_data)}件")
        print("-" * 40)

        if not result_data:
            print("データを取得できませんでした。")
        else:
            print("日付, 始値, 高値, 安値, 終値")
            # 全件表示だと長いので、最初と最後、および全データを確認用に表示
            for item in result_data:
                print(item)

    finally:
        driver.quit()

サイトにアクセスしています...
データ取得開始: 2025-08-02 以前のデータに到達するまで遡ります。
時間がかかります（目標: 120件以上）...
  -> 10件取得 (現在: 2026-01-21)
  -> 20件取得 (現在: 2026-01-06)
  -> 30件取得 (現在: 2025-12-18)
  -> 40件取得 (現在: 2025-12-04)
  -> 50件取得 (現在: 2025-11-19)
  -> 60件取得 (現在: 2025-11-05)
  -> 70件取得 (現在: 2025-10-21)
  -> 80件取得 (現在: 2025-10-06)
  -> 90件取得 (現在: 2025-09-19)
  -> 100件取得 (現在: 2025-09-04)
  -> 110件取得 (現在: 2025-08-21)
画面端に到達しました。
----------------------------------------
スクレイピングにかかった時間: 308.40秒
取得件数: 110件
----------------------------------------
日付, 始値, 高値, 安値, 終値
['2026-02-03', '53,332.18', '54,406.91', '53,307.74', '54,379.93']
['2026-02-02', '53,575.37', '54,247.15', '52,655.18', '52,655.18']
['2026-01-30', '53,434.73', '53,590.24', '52,923.12', '53,322.85']
['2026-01-29', '53,301.26', '53,742.69', '52,990.42', '53,375.6']
['2026-01-28', '53,023.88', '53,507.18', '52,788.1', '53,358.71']
['2026-01-27', '52,847.54', '53,334.03', '52,637.66', '53,333.54']
['2026-01-26', '53,023.28', '53,138.67', '52,656', '52,885.25'